In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('analysisData.csv')
sub = pd.read_csv('scoringData.csv')
df = pd.concat([df,sub])

C:\Users\Sihyun Kwon\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Sihyun Kwon\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
pd.set_option('display.max_columns', None)
df.head()

,access,accommodates,amenities,availability_30,availability_365,availability_60,availability_90,bathrooms,bed_type,bedrooms,beds,calculated_host_listings_count,calendar_last_scraped,calendar_updated,cancellation_policy,city,cleaning_fee,country,country_code,description,experiences_offered,extra_people,first_review,guests_included,has_availability,host_about,host_acceptance_rate,host_has_profile_pic,host_id,host_identity_verified,host_is_superhost,host_listings_count,host_location,host_name,host_neighbourhood,host_picture_url,host_response_rate,host_response_time,host_since,host_thumbnail_url,host_total_listings_count,host_url,host_verifications,house_rules,id,instant_bookable,interaction,is_business_travel_ready,is_location_exact,jurisdiction_names,last_review,last_scraped,latitude,license,listing_url,longitude,market,maximum_nights,medium_url,minimum_nights,monthly_price,name,neighborhood_overview,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,notes,number_of_reviews,picture_url,price,property_type,require_guest_phone_verification,require_guest_profile_picture,requires_license,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month,room_type,scrape_id,security_deposit,smart_location,space,square_feet,state,street,summary,thumbnail_url,transit,weekly_price,xl_picture_url,zipcode
0,"You're welcome to chill in your room, cozy up ...",1,"{Wifi,Kitchen,Heating,""Smoke detector"",""Carbon...",0,0,0,0,1.0,Real Bed,1,1.0,1,2018-03-05,7 months ago,flexible,Queens,NaN,United States,US,"A perfect spot for an intern, student or out o...",none,0,2017-08-02,1,t,I am a published playwright and novelist livin...,NaN,t,142871086,f,f,1,"New York, New York, United States",Emily,Jackson Heights,https://a0.muscache.com/im/pictures/fbfde7c2-f...,NaN,NaN,2017-07-27,https://a0.muscache.com/im/pictures/fbfde7c2-f...,1,https://www.airbnb.com/users/show/142871086,"['email', 'phone', 'facebook', 'google', 'revi...",NaN,20091785,t,I am available via text or email. I'm happy to...,f,t,NaN,2017-08-03,2018-03-05,40.754444,NaN,https://www.airbnb.com/rooms/20091785,-73.895933,New York,1125,NaN,1,NaN,"Cozy Stay in Queens, Easy Access to Manhattan",We're right next to the best Indian food in Qu...,Jackson Heights,East Elmhurst,Queens,NaN,3,https://a0.muscache.com/im/pictures/438180da-d...,45.0,Apartment,f,f,f,9,10,8,10,10,90,10,0.42,Private room,20180303203649,NaN,"Queens, NY",NaN,NaN,NY,"Queens, NY, United States","A perfect spot for an intern, student or out o...",NaN,"Parking is limited, but we're near the E/F/M/R...",NaN,NaN,11370
1,Guests should take there shoes off at the fron...,2,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",3,74,8,24,1.0,Real Bed,1,1.0,3,2018-03-04,3 days ago,moderate,New York,50.0,United States,US,"Nice size apt 15 minutes from central park, a...",none,0,2014-08-11,2,t,"I have been living in New York for 9years, I l...",NaN,t,18930170,t,t,3,"New York, New York, United States",Tom,East Harlem,https://a0.muscache.com/im/pictures/6b4dfca5-4...,100%,within an hour,2014-07-26,https://a0.muscache.com/im/pictures/6b4dfca5-4...,3,https://www.airbnb.com/users/show/18930170,"['email', 'phone', 'reviews', 'kba']",I am asking you to follow these few basic rule...,3710661,t,I can assist with any question you may have re...,f,f,NaN,2018-01-20,2018-03-04,40.785097,NaN,https://www.airbnb.com/rooms/3710661,-73.942337,New York,99,NaN,2,NaN,Spacious room in comfortable apt.,The apartment is conveniently located on the b...,East Harlem,East Harlem,Manhattan,Apartment located on border UES and East H.N...,101,https://a0.muscache.com/im/pictures/57097196/5...,95.0,Apartment,f,f,f,10,10,9,10,8,93,9,2.33,Private room,20180303203649,NaN,"New York, NY",This is a private room in a two bedroom apartm...,NaN,NY,"New York, NY, United States","Nice size apt 15 minutes from central park, a...",NaN,You can take the 6

In [4]:
df.drop(['id', 'experiences_offered','listing_url', 'scrape_id','name', 'summary','space','description','thumbnail_url','medium_url','picture_url',
         'xl_picture_url','host_id','host_url','host_name','host_location','host_thumbnail_url','host_picture_url','host_neighbourhood',
        'street','neighbourhood','neighbourhood_group_cleansed','city','state','market','smart_location','country_code','latitude',
        'longitude','country', 'zipcode'], axis=1, inplace=True)

In [5]:
df.shape

(36428, 65)

### Text columns : T/F based on NaN

In [6]:
str_list = ['neighborhood_overview','notes','transit','access','interaction','house_rules','host_about']

for var in str_list:
    df[var] = df[var].apply(lambda x: 1 if x is np.nan else 0)

### Missing Value over 70%

In [7]:
pd.set_option('display.max_rows', None)
na_df = pd.DataFrame(df.isna().sum()/df.shape[0])

In [8]:
df.drop(list(na_df[na_df[0] > 0.7].index), axis=1, inplace=True)

### Unixtime

In [9]:
import datetime
time_list = ['last_scraped','host_since','calendar_last_scraped']
for var in time_list:
    df[var] = df[var].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').timestamp()).astype(int)

In [10]:
review_list = ['first_review','last_review']    
for var in review_list:
    df[var] = df[var].apply(lambda x: datetime.datetime.strptime(str(x), '%Y-%m-%d').timestamp() if x is not np.NaN else x)

### Oridnal Categoty to Int

In [11]:
def res(x):
    if x =='within an hour':
        return 0
    elif x == 'within a few hours':
        return 1
    elif x == 'within a day':
        return 2
    elif x == 'a few days or more':
        return 3
    else: 
        return x

In [12]:
df['host_response_time'] = df['host_response_time'].apply(lambda x: res(x))

In [13]:
def can_pol(x):
    if x =='flexible':
        return 0
    elif x == 'moderate':
        return 1
    elif x == 'strict':
        return 2
    elif x == 'super_strict_30':
        return 3
    elif x == 'super_strict_60':
        return 3
    else: 
        return x

In [14]:
df['cancellation_policy'] = df['cancellation_policy'].apply(lambda x: can_pol(x))

### Other String Columns

In [15]:
# Remove %
df['host_response_rate'] = df['host_response_rate'].str.strip('%').astype(float)
# df['host_response_rate'] = df['host_response_rate'].astype(float)

In [16]:
# Convert string to int: calnerdar_undated 
cal = pd.DataFrame(list(df['calendar_updated'].apply(lambda x: x.split())), columns=['num','scale','nn'])

In [17]:
def cal_num(x):
    if x =='yesterday':
        return 1
    elif x == 'today':
        return 0
    elif x == 'a':
        return 1
    elif x == 'never':
        return 1000     #I checked a distplot and assigned reasonable number which is high enough
    else: 
        return x

In [18]:
cal['num'] = cal['num'].apply(lambda x: cal_num(x))

In [19]:
def cal_scale(x):
    if x =='months':
        return 30
    elif x == 'days':
        return 1
    elif x == 'week':
        return 7
    elif x == 'weeks':
        return 7
    else: 
        return 1

In [20]:
cal['scale'] = cal['scale'].apply(lambda x: cal_scale(x))

In [21]:
cal['num'] = cal['num'].astype(float)

In [22]:
cal['sum'] = cal['num'] * cal['scale']

In [23]:
df['calendar_updated'] = cal['sum']

In [24]:
# Convert True/False columns
def tf(x):
    if x =='t':
        return 1
    elif x == 'f':
        return 0
    else: 
        return x

In [25]:
tf_list = ['host_is_superhost','host_has_profile_pic','host_identity_verified','is_location_exact','has_availability','requires_license',
          'instant_bookable','is_business_travel_ready','require_guest_profile_picture','require_guest_phone_verification']

In [26]:
for var in tf_list:
    df[var] = df[var].apply(lambda x: tf(x))

In [27]:
# Split List Columns: host_verifications
import re
df['host_verifications'] = df['host_verifications'].apply(lambda x: re.sub('[^A-Za-z0-9,]+', '', x).split(','))

In [28]:
veri = df['host_verifications']
veri_list = []
for i in range(len(veri)):
    veri_list += veri.iloc[i]
    veri_list = list(set(veri_list))    

In [29]:
for i in veri_list:
    df[i] = df['host_verifications'].apply(lambda x: 1 if i in x else 0)

In [30]:
df.drop('host_verifications',axis=1, inplace=True)

In [31]:
# Split List Columns: amenity
df['amenities'] = df['amenities'].apply(lambda x: re.sub('[^A-Za-z0-9,]+', '', x).split(','))

amen = df['amenities']
amen_list = []
for i in range(len(amen)):
    amen_list += amen.iloc[i]
    amen_list = list(set(amen_list)) 

amen_list.remove('')

for i in amen_list:
    df[i] = df['amenities'].apply(lambda x: 1 if i in x else 0)    

df.drop('amenities',axis=1, inplace=True)

### Get Dummy

In [32]:
df = pd.get_dummies(df, columns = ['neighbourhood_cleansed','property_type','room_type','bed_type'], drop_first=True)

In [33]:
df.drop('', axis=1, inplace=True)

### Split data

In [34]:
sub = df[pd.isnull(df['price'])]
df = df[~pd.isnull(df['price'])]

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('price',axis=1), df['price'], test_size=0.3, random_state=83)

### LGBM

In [36]:
import lightgbm

In [37]:
train_data = lightgbm.Dataset(X_train, label=y_train)
test_data = lightgbm.Dataset(X_test, label=y_test)

In [41]:
parameters = {
    'objective': 'tweedie',
    'metric': 'rmse',
    'num_leaves': 15,
    'feature_fraction': 0.3,
    'bagging_fraction': 0.3,
    'bagging_freq': 20,
    'learning_rate': 0.09,
    'max_depth': -1,
    'lambda_l1' : 0.39 ,
    'lambda_l2' : 0.7,
    'tweedie_variance_power': 1.6
}

model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       verbose_eval = 100,
                       early_stopping_rounds=1000)

Training until validation scores don't improve for 1000 rounds.
[100]	valid_0's rmse: 59.2038
[200]	valid_0's rmse: 57.4155
[300]	valid_0's rmse: 56.7599
[400]	valid_0's rmse: 56.8531
[500]	valid_0's rmse: 56.9807
[600]	valid_0's rmse: 56.8004
[700]	valid_0's rmse: 56.7536
[800]	valid_0's rmse: 56.9894
[900]	valid_0's rmse: 57.2463
[1000]	valid_0's rmse: 57.4743
[1100]	valid_0's rmse: 57.5568
[1200]	valid_0's rmse: 57.7236
[1300]	valid_0's rmse: 57.7809
[1400]	valid_0's rmse: 57.7298
[1500]	valid_0's rmse: 57.9097
[1600]	valid_0's rmse: 57.9313
Early stopping, best iteration is:
[678]	valid_0's rmse: 56.601


In [42]:
pred = model.predict(sub)

In [43]:
sub_id = pd.read_csv('scoringData.csv')

In [44]:
output = pd.DataFrame({'id': sub_id['id'], 'price': pred})

In [45]:
output.to_csv("1.csv", index=False)